In [1]:
import os
import pandas as pd


In [2]:
import numpy as np

In [3]:
DATA = "./data"
W2V = './glove'

In [4]:
df = pd.read_csv(os.path.abspath(DATA)+'/imdb_small.csv', sep="|")

In [5]:
msk = np.random.rand(len(df)) < 0.5

In [6]:
df.head(5)

,label,text
0,0,Once again Mr. Costner has dragged out a movie...
1,0,This is an example of why the majority of acti...
2,0,"First of all I hate those moronic rappers, who..."
3,0,Not even the Beatles could write songs everyon...
4,0,Brass pictures (movies is not a fitting word f...


In [7]:
train = df[msk]
test = df[~msk]
X = train.text.as_matrix()
y = train.label.astype(str).as_matrix()
X_test = test.text.as_matrix()
y_test = test.label.astype(str).as_matrix()
print("splitting data to train and test")



splitting data to train and test


In [8]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 120
BATCH_SIZE=2048
EPOCHS=5

In [9]:
from __future__ import print_function

import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Sequential, load_model
from keras.layers import Input, Flatten
from keras.layers import Dense, Activation, Embedding, Dropout
from keras.layers import LSTM, Bidirectional
from keras.optimizers import Adam

from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [10]:
import six.moves.cPickle

In [11]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS) # create dictionary of MAX_NB_WORDS, other words will not be used
tokenizer.fit_on_texts(X)
six.moves.cPickle.dump(tokenizer, open("tokenizer_imdb.pkl", "wb"))

In [12]:
sequences_train = tokenizer.texts_to_sequences(X) # transform words to its indexes
sequences_test = tokenizer.texts_to_sequences(X_test) # transform words to its indexes

In [13]:
word_index = tokenizer.word_index # dictionary of word:index
print('Found %s unique tokens.' % len(word_index))

Found 90889 unique tokens.


In [14]:
data_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data_train.shape)


Shape of data tensor: (25104, 120)


In [15]:
labels_train = to_categorical(np.asarray(y))
labels_test = to_categorical(np.asarray(y_test))
print('Shape of label tensor:', labels_train.shape)
print('Shape of label tensor:', labels_test.shape)

Shape of label tensor: (25104, 2)
Shape of label tensor: (24896, 2)


In [16]:
import codecs
embeddings_index = {}
f = codecs.open(os.path.abspath(W2V)+'/glove.6B.300d.txt', 'r', "utf_8_sig")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [17]:
# prepare embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
print(nb_words)
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
       
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

90890


In [30]:
def data_generator():
    l = data_train.shape[0]
    for i in range(1,l):
        yield data_train[i*30:(i+1)*30], data_test[i*30:(i+1)*30]

In [20]:
print('Building model...')
model = Sequential()
#model.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
model.add(embedding_layer)
#model.add(Embedding(20000, 300, input_length=120))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy'])




Building model...


In [21]:
data_train = data_train.reshape(25101, 120)
labels_train =  labels_train.reshape(25101, 2)

ValueError: total size of new array must be unchanged

In [22]:
print('Train...')
model.fit(data_train,labels_train, batch_size=100, nb_epoch=EPOCHS, show_accuracy=True)


score, acc = model.evaluate(data_test, labels_test,
                            batch_size=BATCH_SIZE)

print("score, acc")
print("%.4f\t%.4f"%(score, acc))


print("Saving model")
plot_model(model, to_file='model.png')
model.save("rt-model.h5")
print("done")

Train...


C:\Users\Alex\Anaconda3\lib\site-packages\keras\models.py:654: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Epoch 1/5
 9700/25104 [==========>...................] - ETA: 121s - loss: 0.6734 - acc: 0.5797

KeyboardInterrupt: 

In [118]:

print("Saving model")
model.save("rt-model.h5")
print("done")

Saving model
done
